In [10]:
import slate
import urllib
import re

In [3]:
furl = urllib.urlopen('http://www.nature.com/ismej/journal/v10/n1/pdf/ismej2015100a.pdf')
with open( '/tmp/asdfasdfasdf.pdf', 'w' ) as ftempfile :
    ftempfile.write( furl.read() )
with open( '/tmp/asdfasdfasdf.pdf' ) as f :
    doc = slate.PDF(f)

In [19]:
#urlre = re.compile( '''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''' )
urlre = re.compile( '(?P<url>https?://[^\s]+)' )

for page in doc :
    print urlre.findall( page.replace('\n','') )

[]
['http://prodege.jgi-psf.org//downloads/availablerun', 'http://prodege.jgi-psf.org,which']
[]
['http://img.jgi.Cell', 'http://creativecommons.org/licenses/by/4.0/the', 'http://www.nature.com/ismej)The']
